In [21]:
from __future__ import print_function
import pandas as pd
import numpy as np
import csv
import re
from keras.callbacks import LambdaCallback
import numpy as np
import random
import sys
import io

In [22]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras import optimizers
from keras.optimizers import RMSprop

In [23]:
def func(df,max_len_synopsis):
    fin_text=df.iloc[0]['synopsis']
    max_len_synopsis = max(max_len_synopsis, len(fin_text))
    chars_temp = set(fin_text)
    #chars.union(chars_temp)
    for review in df.iloc[0]['reviews']:
        chars_temp=chars_temp.union(set(review))
    return chars_temp,max_len_synopsis

In [24]:
from ast import literal_eval

In [25]:
chunksize = 1
fin_text=""
chars = set(fin_text)
max_len_synop=0
for chunk in pd.read_csv('preprocessed.tsv', chunksize=chunksize, delimiter='\t'):
    df= pd.DataFrame(chunk)
    df['reviews'] = df['reviews'].apply(literal_eval)
    temp=func(df,max_len_synop)
    chars = chars.union(temp[0])
    max_len_synop=max(max_len_synop,temp[1])
chars_tuple = tuple(chars)
int2char = dict(enumerate(chars_tuple))
char2int = {ch: ii for ii, ch in int2char.items()}

In [26]:
print(chars)
print(max_len_synop)

{'m', '.', ')', 'K', 'U', '1', 'i', ':', 'o', 's', 'j', '$', ',', 'M', 'T', 'u', 'B', 'é', 'n', 'Z', 'p', 'k', 'G', '4', 'R', '\n', 'y', '(', 'F', 'W', '3', 'O', 'X', '2', 'H', 'L', 'h', '8', '\x96', 'I', 'ê', '&', 'x', '~', '5', ' ', 'E', 'V', 'e', '0', '%', 'S', 'g', '`', "'", 'w', 'f', '-', '*', 'z', 'r', 'C', '[', '?', 'v', '6', 'J', 't', 'c', '=', '"', '/', 'b', 'l', 'q', 'Y', '9', '7', 'a', '!', '+', 'Q', 'A', ';', 'N', '#', 'D', 'P', ']', 'd'}
11801


In [27]:
#df=pd.read_csv('preprocessed.tsv',delimiter='\t', nrows=3)

In [28]:
len_of_review = len(df.iloc[0]['reviews'][0])
fortyforty = (int)((len_of_review -35)/5)


In [29]:

global_review = ""
global_synop = ""
global_rating = ""

In [30]:
# max_len_synopsis=0
# max_len_reviews=0 
# no_of_reviews=0
# tot_no_of_reviews=0
# fin_text=''
# for index,row in df.iterrows():
#     max_len_synopsis = max(max_len_synopsis, len(row['synopsis']))
#     text=''.join(row['synopsis'])
#     fin_text+=' '+text
# for i in range(0, len(df)):
#     temp = df.iloc[i]['reviews']
#     no_of_reviews=0;
#     for text in temp:
#         max_len_reviews = max(max_len_reviews, len(text))
#         no_of_reviews=no_of_reviews+1
#         fin_text+=' '+text
#     tot_no_of_reviews += no_of_reviews
# print(tot_no_of_reviews)
# chars = tuple(set(fin_text))
# int2char = dict(enumerate(chars))
# char2int = {ch: ii for ii, ch in int2char.items()}
# fortyforty = (int)((max_len_reviews-35)/5)

# #print(fortyforty)
# max_len_reviews = 36+5*fortyforty
# #print(max_len_reviews)

In [31]:
def one_hot_encoder(synopsis, rev40,rating):
#     print(max_len_synop)
    encoded = np.array([char2int[ch] for ch in synopsis])
    encoded2 = np.array([char2int[ch] for ch in rev40])
    ans = np.zeros((max_len_synop+40, len(int2char)+1), dtype = np.float32)
    cnt=max_len_synop-len(synopsis)
    for x in encoded:
#         print(x)
        ans[cnt][x] = 1.0
        cnt=cnt+1
    for x in encoded2:
#         print(x)
        ans[cnt][x] = 1.0
        cnt=cnt+1
    temp=0;
    rating_fin=""
    while(rating[temp]!='/'):
        rating_fin+=rating[temp]
        temp=temp+1
    for i in range(max_len_synop+40):
        ans[i][len(int2char)]=(float)(rating_fin)
    return ans

In [32]:
# def one_hot_encoder(arr, n_labels):
    
#     # Initialize the the encoded array
#     one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
#     # Fill the appropriate elements with ones
#     one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
#     # Finally reshape it to get back to the original array
#     one_hot = one_hot.reshape((*arr.shape, n_labels))
    
#     return one_hot

In [33]:
def char_encoding2arr(c):
    encoded = np.zeros(len(int2char), dtype = np.float32)
    encoded[char2int[c]]=1.0
    return encoded

In [34]:

model = Sequential()
model.add(LSTM(128, input_shape=(40+max_len_synop, len(int2char)+1)))
model.add(Dense(len(int2char)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [35]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(global_review) - 40- 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = global_review[start_index: start_index +40]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(300):
            x_pred = np.zeros((1, 40+max_len_synop, len(int2char)+1))
#             for t, char in enumerate(sentence):
#                 x_pred[0, t+max_len_synop, char2int[char]] = 1.
            
            x_pred[0] = one_hot_encoder(global_synop, sentence , global_rating)
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = int2char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [36]:
from keras.callbacks import ModelCheckpoint

filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',verbose=1, save_best_only=True,mode='min')

In [37]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,patience=1, min_lr=0.001)
callbacks = [print_callback, checkpoint, reduce_lr]

In [38]:
def ainvay():
    print(len(global_review))
    print(len(global_synop))

In [39]:
def machau(synopsis,review,rating):
    fortyforty = (len(review) -36)//5
    Y = np.zeros((fortyforty, len(int2char)), dtype=np.float32)
    X = np.zeros((fortyforty, 40+max_len_synop, len(int2char)+1), dtype = np.float32)
    for k in range(40, len(review), 5):
        l = (int)((k-40)/5)
        Y[l] = char_encoding2arr(review[k])
        
    for i in range(fortyforty):
        start_index=i*5;
        X[i]=one_hot_encoder(synopsis,review[start_index:start_index+40],rating)
    global global_review 
    global global_synop
    global global_rating
    global_review = review
    global_synop = synopsis
    global_rating =rating
    print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
    filepath = "weights.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='loss',verbose=1, save_best_only=True,mode='min')
    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,patience=1, min_lr=0.001)
    callbacks = [print_callback, checkpoint, reduce_lr]
    ainvay()
#     print(len(global_review))
    model.fit(X , Y , batch_size=3 , epochs=3 ,callbacks=callbacks)
#     print(Y[0])
#     print(X[0])

In [40]:
chunksize = 1
for chunk in pd.read_csv('preprocessed.tsv', chunksize=chunksize, delimiter='\t'):
    df= pd.DataFrame(chunk)
    df['reviews'] = df['reviews'].apply(literal_eval)
    df['user_ratings'] = df['user_ratings'].apply(literal_eval)
    index=0
    for review in df.iloc[0]['reviews']:
        machau(df.iloc[0]['synopsis'],review,df.iloc[0]['user_ratings'][index])
        index=index+1
        break
    break 

1734
11801
Epoch 1/3
339/339 [==============================] - 541s 2s/step - loss: 3.3020

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "ul. It takes a movie like The Shawshank "
ul. It takes a movie like The Shawshank    e  h h   he h  h   e ee h  t h   ee e  e   he     dheh e hh e e t  e   od   h th e e h e   h hh  h  e d e eh e   h et t h t  e  eeh the  e h     e e

KeyboardInterrupt: 

In [ ]:
def generate_text(length, diversity,review,synopsis,rating):
    # Get random starting text
    start_index = random.randint(0, len(review) - 40 - 1)
    generated = ''
    sentence = review[start_index: start_index + 40]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, 40+max_len_synop, len(int2char)))
#             for t, char in enumerate(sentence):
#                 x_pred[0, t+max_len_synop, char2int[char]] = 1.
            x_pred[0] = one_hot_encoder(synopsis, sentence,rating)
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = int2char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

    return generated

In [ ]:
print(generate_text(300, 0.2,df.iloc[0]['reviews'][0] ,df.iloc[0]['synopsis'],df.iloc[0]['user_ratings'][0]))